# Titanic-survivor

### Importing all neccessary libraries
these libraries helps us perform various tasks like linear algebra and preprocessing and would also be used to train our model and predict using it

In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# sklearn libraries has various in-built classes which consists of various models
# like Linear Regression,Random Forest etc.
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier

# xgboost is another model that implements boosting algorithm
# XGBoost is an implementation of gradient boosted decision trees 
# designed for speed and performance.
import xgboost as xgb

#### creating pandas DataFrame from train and test data

In [2]:
train = pd.read_csv('./titanic_train.csv')
test = pd.read_csv('./titanic_test.csv')
train.head()

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


*The train data consists of 12 columns like PassengerId,Survived,Pclass etc*

*The train data consists of 11 columns all like the train data except Survived which we have to predict.*

#### Many a time various columns do not any major significance in the class we are predicting
#### So we drop those columns in both train and test DataFrame.

In [3]:
train = train.drop(['PassengerId','Ticket','Cabin','Fare'], axis = 1)
test = test.drop(['PassengerId','Ticket','Cabin','Fare'],axis = 1)
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,S
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,S
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,S


In [4]:
#printing the datatypes of all features in train
train.dtypes

Survived      int64
Pclass        int64
Name         object
Sex          object
Age         float64
SibSp         int64
Parch         int64
Embarked     object
dtype: object

In [5]:
# printing the number of null values that each feature in train has
train.isnull().sum()

Survived      0
Pclass        0
Name          0
Sex           0
Age         177
SibSp         0
Parch         0
Embarked      2
dtype: int64

In [6]:
# printing the number of null values that each feature in test has
test.isnull().sum()

Pclass       0
Name         0
Sex          0
Age         86
SibSp        0
Parch        0
Embarked     0
dtype: int64

***From the above we can conclude that there is a huge number of null values in the age feature removing which would be the biggest challenge during cleaning***.

## Cleaning the Data

**Approach**

Taking a look at the data we can see that the feature name(which has no null value) has a format ie. last_name, title. first_name.

So we can use the title of the passengers with known ages to predict the ages of the unknown with the same titles.

In [7]:
# defining function name_prefix that picks up the title of the person from its name and return it.

# the titles given in the function are not the only titles we have, there are many more but their
# number is too insignificant for us to consider.(we will see this later when we groupby the dataset
# according to the label)


def name_prefix(value):
    if ' Mr. ' in value:
        return 'Mr.'
    elif ' Mrs. ' in value:
        return 'Mrs.'
    elif ' Miss. ' in value:
        return 'Miss'
    elif 'Master. ' in value:
        return 'Master'
    elif ' Dr. ' in value:
        return 'Dr.'

**using apply function to apply the name_prefix function to the name column of the dataset and store it as a new feature name_prefix in both train and test data.**

In [8]:
train['name_prefix'] = train['Name'].apply(name_prefix)   
train.head()

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,name_prefix
0,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,S,Mr.
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,C,Mrs.
2,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,S,Miss
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,S,Mrs.
4,0,3,"Allen, Mr. William Henry",male,35.0,0,0,S,Mr.


In [9]:
test['name_prefix'] = test['Name'].apply(name_prefix)   
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,name_prefix
0,3,"Kelly, Mr. James",male,34.5,0,0,Q,Mr.
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,S,Mrs.
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,Q,Mr.
3,3,"Wirz, Mr. Albert",male,27.0,0,0,S,Mr.
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,S,Mrs.


In [10]:
train.dtypes

Survived         int64
Pclass           int64
Name            object
Sex             object
Age            float64
SibSp            int64
Parch            int64
Embarked        object
name_prefix     object
dtype: object

In [11]:
train['name_prefix'].isnull().sum()

20

In [12]:
train[train['name_prefix'].isnull()].isnull().sum()

Survived        0
Pclass          0
Name            0
Sex             0
Age             0
SibSp           0
Parch           0
Embarked        0
name_prefix    20
dtype: int64

***From the above we can conclude that there are 20 rows/datapoints for which we do not have a title. But on the same hand we do not have any datapoint for which both Age and name_prefix are not known.***

*so those 5 titles were enough for us to predict the ages of the not known in the train data.*

In [13]:
train.groupby('name_prefix')['Age'].describe()

,count,mean,std,min,25%,50%,75%,max
name_prefix,,,,,,,,
Dr.,6.0,42.000000,12.016655,23.00,35.000,46.5,49.75,54.0
Master,36.0,4.574167,3.619872,0.42,1.000,3.5,8.00,12.0
Miss,146.0,21.773973,12.990292,0.75,14.125,21.0,30.00,63.0
Mr.,398.0,32.368090,12.708793,11.00,23.000,30.0,39.00,80.0
Mrs.,108.0,35.898148,11.433628,14.00,27.750,35.0,44.00,63.0


In [14]:
# using mean and groupby functions to get the mean age of all people with known
# ages in each title category.

mean_age_train = train.groupby('name_prefix')['Age'].mean()

In [15]:
mean_age_train

name_prefix
Dr.       42.000000
Master     4.574167
Miss      21.773973
Mr.       32.368090
Mrs.      35.898148
Name: Age, dtype: float64

In [16]:
mean_age_train['Mr.']

32.368090452261306

*printing all the datapoints with null ages as a reference to after we clean all of them.*

In [17]:
train[train['Age'].isnull()]

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,name_prefix
5,0,3,"Moran, Mr. James",male,NaN,0,0,Q,Mr.
17,1,2,"Williams, Mr. Charles Eugene",male,NaN,0,0,S,Mr.
19,1,3,"Masselmani, Mrs. Fatima",female,NaN,0,0,C,Mrs.
26,0,3,"Emir, Mr. Farred Chehab",male,NaN,0,0,C,Mr.
28,1,3,"O'Dwyer, Miss. Ellen ""Nellie""",female,NaN,0,0,Q,Miss
29,0,3,"Todoroff, Mr. Lalio",male,NaN,0,0,S,Mr.
31,1,1,"Spencer, Mrs. William Augustus (Marie Eugenie)",female,NaN,1,0,C,Mrs.
32,1,3,"Glynn, Miss. Mary Agatha",female,NaN,0,0,Q,Miss
36,1,3,"Mamee, Mr. Hanna",male,NaN,0,0,C,Mr.
42,0,3,"Kraeff, Mr. Theodor",male,NaN,0,0,C,Mr.


In [18]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 891 entries, 0 to 890
Data columns (total 9 columns):
Survived       891 non-null int64
Pclass         891 non-null int64
Name           891 non-null object
Sex            891 non-null object
Age            714 non-null float64
SibSp          891 non-null int64
Parch          891 non-null int64
Embarked       889 non-null object
name_prefix    871 non-null object
dtypes: float64(1), int64(4), object(4)
memory usage: 62.7+ KB


In [19]:
# defining function age_fill 

# this function checks if the age in the dataset is NaN, if yes it checks for the the title 
# in the name_prefix columns and then asigns its corrosponding value in the mean_age_train
# dictionary as the person's Age. 

def fill_age(value):
    if value['name_prefix'] == 'Mr.' and np.isnan(value['Age']):
        value['Age'] = mean_age_train['Mr.']
    elif value['name_prefix'] == 'Mrs.' and np.isnan(value['Age']):
        value['Age'] = mean_age_train['Mrs.']
    elif value['name_prefix'] == 'Master' and np.isnan(value['Age']):
        value['Age'] = mean_age_train['Master']
    elif value['name_prefix'] == 'Miss' and np.isnan(value['Age']):
        value['Age'] = mean_age_train['Miss']
    elif value['name_prefix'] == 'Dr.' and np.isnan(value['Age']):
        value['Age'] = mean_age_train['Dr.']
    return value

In [20]:
# using the apply funtion to fill in the NaN values in the Age column of the training data.

train_new = train.apply(fill_age,axis = 1)

In [21]:
#new train data with all NaN values in Age filled with the corrosponding mean value of its name_prfix group

train_new

,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,name_prefix
0,0,3,"Braund, Mr. Owen Harris",male,22.000000,1,0,S,Mr.
1,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.000000,1,0,C,Mrs.
2,1,3,"Heikkinen, Miss. Laina",female,26.000000,0,0,S,Miss
3,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.000000,1,0,S,Mrs.
4,0,3,"Allen, Mr. William Henry",male,35.000000,0,0,S,Mr.
5,0,3,"Moran, Mr. James",male,32.368090,0,0,Q,Mr.
6,0,1,"McCarthy, Mr. Timothy J",male,54.000000,0,0,S,Mr.
7,0,3,"Palsson, Master. Gosta Leonard",male,2.000000,3,1,S,Master
8,1,3,"Johnson, Mrs. Oscar W (Elisabeth Vilhelmina Berg)",female,27.000000,0,2,S,Mrs.
9,1,2,"Nasser, Mrs. Nicholas (Adele Achem)",female,14.000000,1,0,C,Mrs.


In [22]:
train_new.isnull().sum()

Survived        0
Pclass          0
Name            0
Sex             0
Age             0
SibSp           0
Parch           0
Embarked        2
name_prefix    20
dtype: int64

**We are now left with only 2 null values in the Embarked feature, which we'll handle later.**

Note : we do not need to worry about name_prefix as it was only made for filling the unknown ages and would be dropped later.

#### Applying the same cleaning process to the test data.

In [23]:
test.head()

,Pclass,Name,Sex,Age,SibSp,Parch,Embarked,name_prefix
0,3,"Kelly, Mr. James",male,34.5,0,0,Q,Mr.
1,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,S,Mrs.
2,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,Q,Mr.
3,3,"Wirz, Mr. Albert",male,27.0,0,0,S,Mr.
4,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,S,Mrs.


In [24]:
test_new = test.apply(fill_age,axis = 1)

In [25]:
test_new[['Age']].head(15)

,Age
0,34.50000
1,47.00000
2,62.00000
3,27.00000
4,22.00000
5,14.00000
6,30.00000
7,26.00000
8,18.00000
9,21.00000


#### now most of our cleaning process is complete and so is our need of Name and name_prefix columns so we will drop them from both train and test.

In [26]:
train_new.drop(['Name','name_prefix'],axis=1,inplace=True)

# making dummies for the Embarked column
train_new=pd.get_dummies(train_new)
# dropping one of the 3 Embarked columns as two columns are enough to repesent the column pefectly
train_new.drop(['Sex_female','Embarked_Q',],axis=1,inplace=True)
train_new.head(10)

,Survived,Pclass,Age,SibSp,Parch,Sex_male,Embarked_C,Embarked_S
0,0,3,22.00000,1,0,1,0,1
1,1,1,38.00000,1,0,0,1,0
2,1,3,26.00000,0,0,0,0,1
3,1,1,35.00000,1,0,0,0,1
4,0,3,35.00000,0,0,1,0,1
5,0,3,32.36809,0,0,1,0,0
6,0,1,54.00000,0,0,1,0,1
7,0,3,2.00000,3,1,1,0,1
8,1,3,27.00000,0,2,0,0,1
9,1,2,14.00000,1,0,0,1,0


In [27]:
test_new.drop(['Name','name_prefix'],axis=1,inplace=True)
test_new=pd.get_dummies(test_new)

test_new.drop(['Sex_female','Embarked_Q',],axis=1,inplace=True)
test_new.head(10)

,Pclass,Age,SibSp,Parch,Sex_male,Embarked_C,Embarked_S
0,3,34.5,0,0,1,0,0
1,3,47.0,1,0,0,0,1
2,2,62.0,0,0,1,0,0
3,3,27.0,0,0,1,0,1
4,3,22.0,1,1,0,0,1
5,3,14.0,0,0,1,0,1
6,3,30.0,0,0,0,0,0
7,2,26.0,1,1,1,0,1
8,3,18.0,0,0,0,1,0
9,3,21.0,2,0,1,0,1


In [28]:
# checking for any NaN values in the test data
test_new.isnull().sum()

Pclass        0
Age           1
SibSp         0
Parch         0
Sex_male      0
Embarked_C    0
Embarked_S    0
dtype: int64

*We still have one unknown value left in the age column of the train dataset.*

*So we'll simply take the mean of all ages and assign it to the missing value.*

In [29]:
test_new.loc[np.isnan(test_new['Age']),'Age'] = test_new['Age'].mean()
test_new.isnull().sum()

Pclass        0
Age           0
SibSp         0
Parch         0
Sex_male      0
Embarked_C    0
Embarked_S    0
dtype: int64

#### No null values left in training as well as testing data.

## Training and Testing 

In [30]:
#train features
X_train = train_new.drop('Survived',axis=1)
#train labels
y_train = train_new['Survived']

# this is the actual results of the whether the passenger in test survived or not
# we will be using it for scoring the accuracy of our model.
actual = pd.read_csv("../datasets/gender_submission.csv")
actual.head()

,PassengerId,Survived
0,892,0
1,893,1
2,894,0
3,895,0
4,896,1


In [31]:
# dropping the PassengerId
# now we only have the actual survived value corrosponding to each passenger.

actual.drop('PassengerId',axis=1,inplace=True)
actual = actual.values.reshape(-1)
actual

array([0, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 1, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 0, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 1, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

In [32]:
# making a dictionary of several classification models 
dict_clf = {
    'linear regression' : LinearRegression(),
    'KNN' : KNeighborsClassifier(),
    'decision tree' : DecisionTreeClassifier(random_state=42),
    'random forest' : RandomForestClassifier(random_state=18),
    'xgboost' : xgb.XGBClassifier(objective='binary:logistic',n_estimators=10,seed=42)
}

# an accuracy dictionary which we'll be using to store the accuracy of various models.
accuracy = {}

In [33]:
print(test_new.shape)
print(actual.shape)

(418, 7)
(418,)


In [34]:
# running a loop through dict_keys 
for key,value in dict_clf.items():
    #the value of dictionary stores the classifier
    #assigning the classification model to clf
    clf = value
    # fitting our model
    clf.fit(X_train,y_train)
    # making predictions on the test data
    predictions = clf.predict(test_new)
    # checking the accuracy by comparing the predictions with the actual values
    score = clf.score(test_new,actual)
    accuracy[key] = score

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


In [35]:
accuracy

{'linear regression': 0.6753152171190491,
 'KNN': 0.7679425837320574,
 'decision tree': 0.7894736842105263,
 'random forest': 0.8181818181818182,
 'xgboost': 0.868421052631579}

**from this we can conclude that RandomForest and XGBoost gives us the maximum accuracy.**

**So now lets tune the parameters to achieve a better accuracy at these two**

## Tuning XGBoost hyperparameters

In [36]:

# TLDR: You can ignore the warning. It is caused by sklearn doing something
# internally that is not quite ideal.

# The warning is actually caused by numpy, which deprecated truth testing
# on empty arrays:

# The long and short is that truth-testing on empty arrays is dangerous,
# misleading, and not in any way useful, and should be deprecated.



# This might also take some time as the GridSearchCV make every possible
# combination of the given hyperparameters and the train the model.


from sklearn.model_selection import GridSearchCV
# For exhaustive search over specified parameter values for an estimator.


# dictionary of all the parameters that we want to tune as keys and
# the corrosponding set of values for that parameter
clf_param_grid = {
    'max_depth' : range(5,10),
    'learning_rate' : [0.1,0.2,0.3],
    'n_estimators' : [10,15,20,25],
    'gamma' : [1e-1,1e-2],
    'min_child_weight': [1,2,3]
}


estimators = GridSearchCV(estimator = xgb.XGBClassifier(objective = 'binary:logistic',seed=42),
                          param_grid = clf_param_grid,
                          cv=4, 
                          verbose=1,
                          scoring='accuracy')

estimators.fit(X_train,y_train)


/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

Fitting 4 folds for each of 360 candidates, totalling 1440 fits


/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

...

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array 

GridSearchCV(cv=4, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
       colsample_bytree=1, gamma=0, learning_rate=0.1, max_delta_step=0,
       max_depth=3, min_child_weight=1, missing=None, n_estimators=100,
       n_jobs=1, nthread=None, objective='binary:logistic', random_state=0,
       reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=42, silent=True,
       subsample=1),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': range(5, 10), 'learning_rate': [0.1, 0.2, 0.3], 'n_estimators': [10, 15, 20, 25], 'gamma': [0.1, 0.01], 'min_child_weight': [1, 2, 3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [37]:
print(estimators.best_params_) #prints the best parameter combination for the model
print(estimators.best_score_) #print the score using the best parameters

{'gamma': 0.1, 'learning_rate': 0.3, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 15}
0.8395061728395061


#### Some of the results of the grid search

{'learning_rate': 0.3, 'max_depth': 10, 'n_estimators': 10}
**0.8338945005611672**

{'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 20}
**0.8372615039281706**

{'gamma': 0.01, 'learning_rate': 0.2, 'max_depth': 7, 'n_estimators': 20}
**0.8395061728395061**

{'gamma': 0.01, 'learning_rate': 0.2, 'max_depth': 7, 'min_child_weight': 1, 'n_estimators': 20}
**0.8395061728395061**

{'gamma': 0.1, 'learning_rate': 0.3, 'max_depth': 6, 'min_child_weight': 3, 'n_estimators': 15}
**0.8395061728395061**

#### Putting the best values for the parameter and testing the accuracy

In [47]:
clf = xgb.XGBClassifier(objective='binary:logistic',
                        n_estimators=15,
                        max_depth = 6,
                        min_child_weight = 3,
                        learning_rate = 0.3,
                        gamma = 0.1,
                        seed=42)

clf.fit(X_train,y_train)
predictions = clf.predict(test_new)
score = clf.score(test_new,actual)
score

/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:
/home/pratham/anaconda3/lib/python3.6/site-packages/sklearn/preprocessing/label.py:151: DeprecationWarning: The truth value of an empty array is ambiguous. Returning False, but in future this will result in an error. Use `array.size > 0` to check that an array is not empty.
  if diff:


0.8803827751196173

In [39]:
# taking a look at the predictions
predictions

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 0, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 1,
       1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1,
       0, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

## Tuning RandomForest Hyperparameters 

**We will now apply the same parameter tuning process as we did for XGBoost**

In [41]:
clf_param_grid = {
    'max_depth': range(5,21,5),
    'max_features': [2, 3, 4, 5],
    'min_samples_leaf': [3, 4, 5],
    'min_samples_split': [4, 8, 12],
    'n_estimators': [10,20,30,40]
}

estimators = GridSearchCV(estimator = RandomForestClassifier(random_state=42),
                          param_grid = clf_param_grid,
                          cv=4, 
                          verbose=1,
                          scoring='accuracy')

estimators.fit(X_train,y_train)

Fitting 4 folds for each of 576 candidates, totalling 2304 fits


[Parallel(n_jobs=1)]: Done 2304 out of 2304 | elapsed:  1.1min finished


GridSearchCV(cv=4, error_score='raise',
       estimator=RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_leaf_nodes=None,
            min_impurity_decrease=0.0, min_impurity_split=None,
            min_samples_leaf=1, min_samples_split=2,
            min_weight_fraction_leaf=0.0, n_estimators=10, n_jobs=1,
            oob_score=False, random_state=42, verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'max_depth': range(5, 21, 5), 'max_features': [2, 3, 4, 5], 'min_samples_leaf': [3, 4, 5], 'min_samples_split': [4, 8, 12], 'n_estimators': [10, 20, 30, 40]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring='accuracy', verbose=1)

In [42]:
print(estimators.best_params_)
print(estimators.best_score_)

{'max_depth': 5, 'max_features': 2, 'min_samples_leaf': 3, 'min_samples_split': 8, 'n_estimators': 30}
0.8271604938271605


In [43]:
clf = RandomForestClassifier(max_depth=5,max_features=2,min_samples_leaf=3,min_samples_split=8,n_estimators=30,random_state=42)

clf.fit(X_train,y_train)
predictions = clf.predict(test_new)
score = clf.score(test_new,actual)
score

0.9090909090909091

In [44]:
predictions

array([0, 0, 0, 0, 0, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 1, 0, 0,
       1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0,
       1, 1, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 0, 1, 1, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       0, 0, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 1, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 0, 1, 0, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 1, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0,

### So finally we have trained a model which could predict your survival chances in the titanic tragedy with *90.90%* of accuracy using Random Forest Classifer.

*For future references we will store our predictions in a new csv file having the passenderid and the corrosponding survival prediction to it*

In [45]:
passenger_id = 892
col_names = ['PassengerId','Survived']

with open('./titanic.csv','w') as f:
    f.write(col_names[0]+','+col_names[1]+'\n')
    for i in predictions:
        f.write(str(passenger_id)+','+str(i)+'\n')
        passenger_id += 1